In [1]:
import os
import sys
import re
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [2]:
#INPUTS
folder='/processing_Data/antibioticos/sgarciacobos/projects/virulence_kpOXA48_NL_ES/ANALYSIS/mash'

In [7]:
def return_best_matches(screen_file):
    df = pd.read_csv(screen_file, sep='\t', names=['identity', 'shared-hashes', 'median-multiplicity', 'p-value', 'query-ID', 'query-comment'])
    df = df[~df['query-comment'].str.contains(' phage')]
    df = df[~df['query-comment'].str.contains(' Phage')]
    df = df.sort_values(by=['identity'], ascending=False)
    df.reset_index(inplace=True)
    return df[df.identity > 0.9]

In [51]:
return_best_matches('/processing_Data/antibioticos/sgarciacobos/projects/virulence_kpOXA48_NL_ES/ANALYSIS/mash/ISC11/ISC11.screen.winner.tab')

,index,identity,shared-hashes,median-multiplicity,p-value,query-ID,query-comment
0,238,0.999618,992/1000,10,0.0,GCF_000779545.1_ASM77954v1_genomic.fna.gz,"[141 seqs] NZ_JSLT01000002.1 Escherichia coli strain upec-193 upec-193_ctg_2220, whole genome shotgun sequence [...]"
1,984,0.998058,960/1000,34,0.0,GCF_900086545.1_12082_5_28_genomic.fna.gz,"[47 seqs] NZ_FLJZ01000001.1 Klebsiella pneumoniae strain k1206, whole genome shotgun sequence [...]"


In [9]:
df_mash = return_best_matches('/processing_Data/antibioticos/sgarciacobos/projects/virulence_kpOXA48_NL_ES/ANALYSIS/mash/ISC11/ISC11.screen.winner.tab')

In [20]:
df_mash.shape[0]

2

In [110]:
def extract_species(df_line):
    split_query = df_line['query-comment'].split(' ')
    if re.match(r'^\[.{4,10}\]', df_line['query-comment']):
        return ' '.join(split_query[3:5])
    else:
        return ' '.join(split_query[1:3])

def report_contamination(df_mash):
    '''
    Dependencies:   -df_mash
                    -extract_species(df_line)
    '''
    main_species = extract_species(df_mash.iloc[0])
    contamination_report = "Main species: " + main_species + "\n"
    if df_mash.shape[0] > 1:
        for index, data_row in df_mash.iloc[1:].iterrows():
            hashes = data_row['shared-hashes'] #.split('/')[0]
            contamined_species = extract_species(data_row)
            if contamined_species != main_species:
                contamination_report = contamination_report + 'Contamination: ' + str(hashes) + " " + contamined_species + "\n"
    return contamination_report

In [111]:
report_contamination(df_mash)

'Main species: Escherichia coli\nContamination: 960/1000 Klebsiella pneumoniae\n'

In [114]:
def find_contamination_mash(folder):
    '''
    Dependencies:   -input folder
                    -def return_best_matches(screen_file)
                    -report_contamination(df_mash)
    '''
    
    #Find common reference from mash result
    for root, _, files in os.walk(folder):
        for name in files:
            if name.endswith(".winner.tab"):
                filename = os.path.join(root, name)
                sample = name.split('.')[0]
                best_matches = return_best_matches(filename)
                return sample + "\n" + report_contamination(best_matches)

In [115]:
find_contamination_mash(folder)

'544651_Kpn\nMain species: Klebsiella pneumoniae\n'